# AirSeaFlux: bulkformulae Optimization Problem

The notebook `optim_and_enzyme.ipynb` includes several examples the use Enzyme and Optim to get adjoints and to minimize some cost function. The input in most cases is a vector, but the output of the function we are optimizing is scalar (a single term from the bulkformulae function). Here we are working to build a version that uses all outputs of bulkformulae.

In [1]:
using Pkg; Pkg.add(url="https://github.com/eldavenport/ECCO.jl"); Pkg.add("AirSeaFluxes")
using ECCO
import AirSeaFluxes: bulkformulae
using Enzyme, Optim

    Updating git-repo `https://github.com/eldavenport/ECCO.jl`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [12]:
#= Output of the bulkformulae function

out = Dict("hl"=>hl,"hs"=>hs,"evap"=>evap,"ch"=>ch,"ce"=>ce,
"tau"=>tau,"ssq"=>ssq,"huol"=>huol,"rd"=>rd,"re"=>re,"rh"=>rh,
"ustar"=>ustar,"qstar"=>qstar,"tstar"=>tstar,"psimh"=>psimh,"psixh"=>psixh)
=#

x0 = [300.0,0.001,1.0,10.0]
obs = [-3.1,2.1,5.5e-9,0.05]

# cost function with the obs as an argument
function J_bulkformulae(x::Vector{Float64},y_obs::Vector{Float64})
    res = bulkformulae(x[1],x[2],x[3],x[4])

    J = abs(res.hl-y_obs[1])^2 + abs(res.hs-y_obs[2])^2 + abs(res.evap-y_obs[3])^2 + abs(res.tau-y_obs[4])^2
    return J
end
# create a new "closure" that will keep y_obs constant
function cost_closure(y_obs::Vector{Float64})
    return x -> J_bulkformulae(x,y_obs)
end

cost = cost_closure(obs)
# cost(x0) = 0.008252726417096927
# cost(x0) 

# get the adjoint of this new closure function that accounts for the constant obs
function cost_ad!(bx2, x) 
    bx = zeros(size(x))
    Enzyme.autodiff(Reverse, cost, Duplicated(x, bx))
    bx2 .= bx
end

# for testing: evaluate the gradient at x0
bx2 = zeros(size(x0))
cost_ad!(bx2,x0)

# optimization with the cost function and it's adjoint 
result=Optim.optimize(cost, cost_ad!, x0)
x1=Optim.minimizer(result)

# # check that tau at x1 is close to y_obs
y1 = bulkformulae(x1[1],x1[2],x1[3],x1[4])
(hl=y1.hl,hs=y1.hs,evap=y1.evap,tau=y1.tau)

(hl = -3.1000000000004873, hs = 2.0999999999998105, evap = 1.240248049610117e-9, tau = 0.04999999999937696)